In [1]:
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate

import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 114.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00


In [2]:
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=7877afff45513dabacc94e6789e59b1b2762cd90cb788f74f981b1c660d39049
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

import json

class PartDataset:
    def __init__(self, dataset, offset, length):
        self.dataset = dataset
        self.offset = offset
        self.length = length
        super(PartDataset, self).__init__()

    def __len__(self):
        return self.length

    def __getitem__(self, i):
        return self.dataset[i + self.offset]

    @staticmethod
    def get_data(input_path):
          dataset = spark.read.format("json").load(input_path)
          return dataset

# Get input path from user
input_path = input("Enter input path: ")

# Load the dataset
dataset = PartDataset.get_data(input_path=input_path)

# Access the first element of the dataset
print(dataset)


Enter input path: /content/drive/MyDrive/Mobin/LLM-Mobin/ENSE3-Schenider/data
DataFrame[message: string]


In [6]:
first_message = dataset.select("message")
filtered_dataset = dataset.filter(dataset["message"].isNotNull())
filtered_dataset.select("message").show()

# Access the "message" key of all rows in the filtered dataset
training_data = [row["message"] for row in filtered_dataset.select("message").collect()]

# Print the list of messages
print(training_data)

+--------------------+
|             message|
+--------------------+
|1. The DS9297 dat...|
|2. The maximum ju...|
|3. With a VCE(sat...|
|4. The STGW25H120...|
|5. These IGBTs su...|
|6. The STGW25H120...|
|7. Additionally, ...|
|You can find more...|
+--------------------+

['1. The DS9297 datasheet provides information on the STGW25H120DF2 and STGWA25H120DF2, which are trench gate field-stop IGBTs from the H series. These components have a voltage rating of 1200 V and a current rating of 25 A, making them suitable for high-speed applications.', '2. The maximum junction temperature for these IGBTs is 175 °C, ensuring reliable operation even under high-temperature conditions. ', '3. With a VCE(sat) of 2.1 V (typical) at a current of 25 A, these IGBTs offer low on-state voltage drop, minimizing power losses in the circuit.', '4. The STGW25H120DF2 and STGWA25H120DF2 IGBTs have a minimum short circuit withstand time of 5 μs at a junction temperature of 150 °C, making them robust and capa

In [7]:
from datasets import Dataset

# Assuming your list is named 'my_list'


# Convert the list to a Hugging Face Dataset
hf_dataset = Dataset.from_dict({"text": training_data})



In [8]:
print(hf_dataset[0])

{'text': '1. The DS9297 datasheet provides information on the STGW25H120DF2 and STGWA25H120DF2, which are trench gate field-stop IGBTs from the H series. These components have a voltage rating of 1200 V and a current rating of 25 A, making them suitable for high-speed applications.'}


In [10]:
# Dataset
#data_name = "mlabonne/guanaco-llama2-1k"
#training_data = load_dataset(data_name, split="train")

# Model and tokenizer names
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
refined_model = "/content/drive/MyDrive/Mobin/LLM-Mobin/ENSE3-Schenider/llama-2-7b-mlabonne-enhanced" #You can give it your own name

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

In [12]:
text_generator = pipeline("text-generation", model=base_model, tokenizer=llama_tokenizer)

# Input text for generation
input_text = "explain me about STGW25H120DF2"

# Generate output
generated_output = text_generator(input_text, max_length=100, num_beams=5, no_repeat_ngram_size=2)

# Print the generated output
print("Input Text:", input_text)
print("Generated Output:", generated_output[0]['generated_text'])

Input Text: explain me about STGW25H120DF2
Generated Output: explain me about STGW25H120DF2L and what it is used for?
 nobody knows about it.

Comment: Please provide more context or information about what you are referring to, so I can better understand and help you. Is it a part number, a product name, or something else? Also, what industry or field are you working in? Any additional details you can provide will help me give you a more accurate answer.


In [13]:
%pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.2 MB/s eta 0:00:00


In [14]:
import wandb
wandb.init()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [15]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

# Training Params
train_params = TrainingArguments(
    output_dir="/content/drive/MyDrive/Mobin/LLM-Mobin/ENSE3-Schenider/results_modified",
    num_train_epochs=100,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb",
    no_cuda = False,

)



# Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=hf_dataset,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params,

)



/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [16]:
print(hf_dataset[0])

{'text': '1. The DS9297 datasheet provides information on the STGW25H120DF2 and STGWA25H120DF2, which are trench gate field-stop IGBTs from the H series. These components have a voltage rating of 1200 V and a current rating of 25 A, making them suitable for high-speed applications.'}


In [17]:
# Training
fine_tuning.train()

# Save Model
fine_tuning.model.save_pretrained(refined_model)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.759500
50,0.198300
75,0.048500
100,0.042700
125,0.042000
150,0.040800
175,0.040900
200,0.040800


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

In [18]:
wandb.finish()

train/epoch,▁▂▃▄▅▆▇██
train/global_step,▁▂▃▄▅▆▇██
train/learning_rate,▁▁▁▁▁▁▁▁
train/loss,█▂▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,100.0
train/global_step,200


In [ ]:
# if we want to use tensorboard instead of weights and biases
# from tensorboard import notebook
# log_dir = "results/runs"
# notebook.start("--logdir {} --port 4000".format(log_dir))


In [19]:
print(refined_model)

/content/drive/MyDrive/Mobin/LLM-Mobin/ENSE3-Schenider/llama-2-7b-mlabonne-enhanced
